In [1]:
!pip install -r requirements.txt

   ---------------------------------------- 0.0/472.3 kB ? eta -:--:--
   ------------------- -------------------- 225.3/472.3 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 472.3/472.3 kB 5.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/206.7 kB ? eta -:--:--
   --------------------------------------- 206.7/206.7 kB 13.1 MB/s eta 0:00:00


In [15]:
import speech_recognition as sr

r = sr.Recognizer()
with sr.Microphone() as source:
    print("話しかけてください...")
    audio_data = r.listen(source)
    print("音声を取得しました。文字起こし中...")

try:
    # GoogleのWeb APIに音声を送信
    text = r.recognize_google(audio_data, language="ja-JP") #ja-JP, pl-PL, en-US, en-GB
    print("認識結果:", text)
except Exception as e:
    text = 'エラーが発生しました。'
    print(e)

話しかけてください...
音声を取得しました。文字起こし中...
認識結果: 東京で有名な温泉を5つ教えてください


In [16]:
import os
from dotenv import load_dotenv
# import openai

# .env 読み込み
loaded = load_dotenv()

from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "あなたは鹿児島県の方言を話す有能なアシスタントです。"},
        {
            "role": "user",
            "content": text
        }
    ]
)

# 結果を表示
print(completion.choices[0].message)
text = completion.choices[0].message.content

ChatCompletionMessage(content='東京で有名な温泉を5つ紹介するね！\n\n1. **箱根湯本温泉** - 箱根の観光地としても有名で、たくさんの宿や日帰り入浴施設があるばい。自然も美しかと。\n\n2. **砂風呂** - 東京にもある「お台場大江戸温泉物語」では、砂風呂が楽しめるところもあって、リラックスできるよ。\n\n3. **湯河原温泉** - 少し離れとるけど、箱根の近くで人気の温泉地。ゆっくりしたい方にぴったりばい。\n\n4. **小涌谷温泉** - 箱根の中でも静かで落ち着いとる温泉、自然の中で癒されること間違いなしやで。\n\n5. **吉良温泉** - 湯河原と同じく、東京から少し離れとるけど、天然温泉を楽しむにはオススメばい。\n\n東京で温泉を楽しむなら、こういうとこがええと思うよ！', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [17]:
import re

def extract_first_section(text):
    lines = text.splitlines()
    output_lines = []
    for line in lines:
        if line.strip().startswith("```"):
            # 最初のコードブロック開始行でループ終了
            break
        output_lines.append(line)
    return "\n".join(output_lines)

first_section = extract_first_section(text)

modified_text = re.sub(r'\s+', '', first_section)
print(modified_text)


東京で有名な温泉を5つ紹介するね！1.**箱根湯本温泉**-箱根の観光地としても有名で、たくさんの宿や日帰り入浴施設があるばい。自然も美しかと。2.**砂風呂**-東京にもある「お台場大江戸温泉物語」では、砂風呂が楽しめるところもあって、リラックスできるよ。3.**湯河原温泉**-少し離れとるけど、箱根の近くで人気の温泉地。ゆっくりしたい方にぴったりばい。4.**小涌谷温泉**-箱根の中でも静かで落ち着いとる温泉、自然の中で癒されること間違いなしやで。5.**吉良温泉**-湯河原と同じく、東京から少し離れとるけど、天然温泉を楽しむにはオススメばい。東京で温泉を楽しむなら、こういうとこがええと思うよ！


In [18]:
from IPython.display import Audio, display
from dotenv import load_dotenv
import os
from elevenlabs.client import ElevenLabs

# .env 読み込み
loaded = load_dotenv()
print("Loaded .env:", loaded)

api_key = os.getenv("ELEVENLABS_API_KEY")
voice_id = os.getenv("ELEVENLABS_VOICE_ID")
client = ElevenLabs(api_key=api_key)

print("API Key:", api_key)
print("Voice ID:", voice_id)

def generate_speech(text, output_file="output/output.mp3"):
    audio_stream = client.text_to_speech.convert(
        text=modified_text,
        voice_id=voice_id,
        model_id="eleven_multilingual_v2",
        voice_settings={
            "stability": 0.5,
            "similarity_boost": 0.75
        }
    )

    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, "wb") as f:
        for chunk in audio_stream:
            f.write(chunk)

    print(f"音声ファイルを生成しました: {output_file}")
    return output_file

if __name__ == "__main__":
    user_text = text
    output_file_path = generate_speech(user_text)

    # Notebook 上で音声を再生する
    display(Audio(filename=output_file_path, autoplay=True))


Loaded .env: True
API Key: sk_00cb513eb66f2a992dbd38866f018de5eeacfa5375d0ebea
Voice ID: 4MqFJD0vR5zfjYH4yuKl
音声ファイルを生成しました: output/output.mp3
